In [1]:
import dedalus.public as d3
import logging

logger = logging.getLogger(__name__)
import copy
import h5py
import numpy as np
import matplotlib

import matplotlib.pyplot as plt
from dedalus.extras import plot_tools
import os
from os import listdir

In [2]:

matplotlib.use('Agg')

# Parameters
Lx, Lz = 4, 1
Nx, Nz = 256, 64

Ra_D = -1.24e5
Prandtl = 0.7
N_s2 = 4

D_0 = 0
D_H = 1
M_0 = 0
M_H = -1

dealias = 3 / 2
stop_sim_time = 20
timestepper = d3.RK222
max_timestep = 0.125
dtype = np.float64

In [3]:
# Bases
coords = d3.CartesianCoordinates('x', 'z')
dist = d3.Distributor(coords, dtype=dtype)
xbasis = d3.RealFourier(coords['x'], size=Nx, bounds=(0, Lx), dealias=dealias)
zbasis = d3.ChebyshevT(coords['z'], size=Nz, bounds=(0, Lz), dealias=dealias)

# Fields
p = dist.Field(name='p', bases=(xbasis, zbasis))
D = dist.Field(name='D', bases=(xbasis, zbasis))
M = dist.Field(name='M', bases=(xbasis, zbasis))
u = dist.VectorField(coords, name='u', bases=(xbasis, zbasis))
Z = dist.Field(name='Z', bases=zbasis)
tau_p = dist.Field(name='tau_p')
tau_B1 = dist.Field(name='tau_B1', bases=xbasis)
tau_B2 = dist.Field(name='tau_B2', bases=xbasis)
tau_D1 = dist.Field(name='tau_D1', bases=xbasis)
tau_D2 = dist.Field(name='tau_D2', bases=xbasis)
tau_M1 = dist.Field(name='tau_M1', bases=xbasis)
tau_M2 = dist.Field(name='tau_M2', bases=xbasis)
tau_u1 = dist.VectorField(coords, name='tau_u1', bases=xbasis)
tau_u2 = dist.VectorField(coords, name='tau_u2', bases=xbasis)


In [4]:
# Substitutions
kappa = (Ra_D * Prandtl / ((D_0 - D_H) * Lz ** 3)) ** (-1 / 2)
nu = (Ra_D / (Prandtl * (D_0 - D_H) * Lz ** 3)) ** (-1 / 2)

# Kuo_Bretherton Equilibrium

# Ra_M
Ra_M = Ra_D * (M_0 - M_H) / (D_0 - D_H)
G_D = (D_0 - D_H) / Lz
G_M = (M_0 - M_H) / Lz
Ra_BV = N_s2 * Lz ** 4 / (nu * kappa)
print(Ra_M)
print(Ra_BV)

x, z = dist.local_grids(xbasis, zbasis)
Z['g'] = z
Z.change_scales(3 / 2)

ex, ez = coords.unit_vector_fields(dist)
lift_basis = zbasis.derivative_basis(1)
lift = lambda A: d3.Lift(A, lift_basis, -1)

B_op = (np.absolute(D - M - N_s2 * Z) + M + D - N_s2 * Z) / 2

Max = lambda A, B: (abs(A - N_s2 * Z - B) + A - N_s2 * Z + B) / 2
eva = lambda A: A.evaluate()

uz = u @ ez

grad_u = d3.grad(u) + ez * lift(tau_u1)  # First-order reduction
grad_M = d3.grad(M) + ez * lift(tau_M1)  # First-order reduction
grad_D = d3.grad(D) + ez * lift(tau_D1)  # First-order reduction


124000.0
496000.00000000006


In [5]:
# Problem
# First-order form: "div(f)" becomes "trace(grad_f)"
# First-order form: "lap(f)" becomes "div(grad_f)"
problem = d3.IVP([p, M, D, u, tau_p, tau_M1, tau_M2, tau_D1, tau_D2, tau_u1, tau_u2], namespace=locals())
problem.add_equation("trace(grad_u) + tau_p= 0")
problem.add_equation("dt(M) - kappa*div(grad_M) + lift(tau_M2) - G_M*uz= - u@grad(M)")
problem.add_equation("dt(D) - kappa*div(grad_D) + lift(tau_D2) - G_D*uz= - u@grad(D)")
problem.add_equation("dt(u) - nu*div(grad_u) + grad(p)  + lift(tau_u2) = - u@grad(u)+ B_op*ez")
problem.add_equation("M(z=0) = M_0")
problem.add_equation("D(z=0) = D_0")
problem.add_equation("u(z=0) = 0")
problem.add_equation("M(z=Lz) = M_H")
problem.add_equation("D(z=Lz) = D_H")
problem.add_equation("u(z=Lz)= 0")
problem.add_equation("integ(p) = 0") # Pressure gauge

{'LHS': Integrate(Integrate(<Field 22889431447728>)),
 'RHS': 0,
 'condition': 'True',
 'tensorsig': (),
 'dtype': numpy.float64,
 'M': 0,
 'L': Integrate(Integrate(<Field 22889431447728>)),
 'F': <Field 22889431311520>,
 'domain': <dedalus.core.domain.Domain at 0x14d15c6e0eb0>,
 'matrix_dependence': array([ True,  True]),
 'matrix_coupling': array([False,  True])}

In [6]:
# Solver
solver = problem.build_solver(timestepper)
solver.stop_sim_time = stop_sim_time

# Initial condition
D.fill_random('g', seed=42, distribution='normal', scale=1e-3) # Random noise
D['g'] *= z * (Lz - z) # Damp noise at walls
D['g'] += Lz - z # Add linear background
M.fill_random('g', seed=42, distribution='normal', scale=1e-3) # Random noise
M['g'] *= z * (Lz - z) # Damp noise at walls
M['g'] += Lz - z # Add linear background

# Analysis
snapshots = solver.evaluator.add_file_handler('snapshots', sim_dt=0.25, max_writes=50)
snapshots.add_task(M, name='moist buoyancy')
snapshots.add_task(D, name='dry buoyancy')
snapshots.add_task(-d3.div(d3.skew(u)), name='vorticity')


2023-09-13 17:09:54,397 subsystems 0/1 INFO :: Building subproblem matrices 1/128 (~1%) Elapsed: 0s, Remaining: 24s, Rate: 5.3e+00/s
2023-09-13 17:09:55,220 subsystems 0/1 INFO :: Building subproblem matrices 13/128 (~10%) Elapsed: 1s, Remaining: 9s, Rate: 1.3e+01/s
2023-09-13 17:09:56,104 subsystems 0/1 INFO :: Building subproblem matrices 26/128 (~20%) Elapsed: 2s, Remaining: 7s, Rate: 1.4e+01/s
2023-09-13 17:09:56,992 subsystems 0/1 INFO :: Building subproblem matrices 39/128 (~30%) Elapsed: 3s, Remaining: 6s, Rate: 1.4e+01/s
2023-09-13 17:09:57,882 subsystems 0/1 INFO :: Building subproblem matrices 52/128 (~41%) Elapsed: 4s, Remaining: 5s, Rate: 1.4e+01/s
2023-09-13 17:09:58,767 subsystems 0/1 INFO :: Building subproblem matrices 65/128 (~51%) Elapsed: 5s, Remaining: 4s, Rate: 1.4e+01/s
2023-09-13 17:09:59,657 subsystems 0/1 INFO :: Building subproblem matrices 78/128 (~61%) Elapsed: 5s, Remaining: 3s, Rate: 1.4e+01/s
2023-09-13 17:10:00,542 subsystems 0/1 INFO :: Building subprob

In [7]:
# CFL
CFL = d3.CFL(solver, initial_dt=max_timestep, cadence=10, safety=0.5, threshold=0.05,
             max_change=1.5, min_change=0.5, max_dt=max_timestep)
CFL.add_velocity(u)

# Flow properties
flow = d3.GlobalFlowProperty(solver, cadence=10)
flow.add_property(np.sqrt(u@u)/nu, name='Re')

# Main loop
startup_iter = 10
try:
    logger.info('Starting main loop')
    while solver.proceed:
        timestep = CFL.compute_timestep()
        solver.step(timestep)
        if (solver.iteration-1) % 10 == 0:
            max_Re = flow.max('Re')
            logger.info('Iteration=%i, Time=%e, dt=%e, max(Re)=%f' %(solver.iteration, solver.sim_time, timestep, max_Re))
except:
    logger.error('Exception raised, triggering end of main loop.')
    raise
finally:
    solver.log_stats()


2023-09-13 17:10:06,775 __main__ 0/1 INFO :: Starting main loop
2023-09-13 17:10:08,012 __main__ 0/1 INFO :: Iteration=1, Time=1.250000e-01, dt=1.250000e-01, max(Re)=0.000000
2023-09-13 17:10:08,888 __main__ 0/1 INFO :: Iteration=11, Time=1.375000e+00, dt=1.250000e-01, max(Re)=0.018661
2023-09-13 17:10:09,756 __main__ 0/1 INFO :: Iteration=21, Time=2.625000e+00, dt=1.250000e-01, max(Re)=0.058826
2023-09-13 17:10:10,607 __main__ 0/1 INFO :: Iteration=31, Time=3.875000e+00, dt=1.250000e-01, max(Re)=0.255093
2023-09-13 17:10:11,568 __main__ 0/1 INFO :: Iteration=41, Time=5.125000e+00, dt=1.250000e-01, max(Re)=1.241694
2023-09-13 17:10:12,426 __main__ 0/1 INFO :: Iteration=51, Time=6.375000e+00, dt=1.250000e-01, max(Re)=6.218802
2023-09-13 17:10:13,283 __main__ 0/1 INFO :: Iteration=61, Time=7.625000e+00, dt=1.250000e-01, max(Re)=30.589609
2023-09-13 17:10:14,145 __main__ 0/1 INFO :: Iteration=71, Time=8.875000e+00, dt=1.250000e-01, max(Re)=155.306080
2023-09-13 17:10:15,410 __main__ 0/1 I

In [8]:

folder_dir = "snapshots"

file_paths = [os.path.join(folder_dir, file) for file in listdir(folder_dir) if os.path.isfile(os.path.join(folder_dir, file)) and file.endswith('.h5')]
file_paths.sort()
print(file_paths)

n = 0
if not os.path.exists('buoyancygraph'):
    os.mkdir('buoyancygraph')

for file in file_paths:
    with h5py.File(file, mode='r') as file:
        moistbuoyancy = file['tasks']['moist buoyancy']
        drybuoyancy = file['tasks']['dry buoyancy']
        buoyancy = np.maximum(moistbuoyancy, drybuoyancy - N_s2 * z)
        st = file['scales/sim_time']
        simtime = np.array(st)
        for t in range(0, len(simtime)):
            mb = np.transpose(moistbuoyancy[t, :, :])
            db = np.transpose(drybuoyancy[t, :, :])
            bu = np.transpose(buoyancy[t, :, :])
            plt.contourf(bu, cmap='RdBu_r')
            plt.colorbar(label='buoyancy')
            plt.xlabel('x')
            plt.ylabel('z')
            n = n + 1
            # Add time title
            title = "t=" + str(simtime[t])
            plt.title(title)
            plt.savefig('buoyancygraph/buoyancy_' + "%04d" % n + '.png', dpi=200, bbox_inches='tight')
            matplotlib.pyplot.close()

['snapshots/snapshots_s1.h5', 'snapshots/snapshots_s2.h5']


In [9]:
import matplotlib.animation as Animation
from PIL import Image

folder_dir = 'buoyancygraph'
# Get a list of PNG file paths from the directory
file_paths = [os.path.join(folder_dir, f) for f in listdir(folder_dir) if
              os.path.isfile(os.path.join(folder_dir, f)) and f.endswith('.png')]

# Sort the list of file paths
file_paths.sort()
# Read the images using PIL
imgs = [Image.open(f) for f in file_paths]
fig = plt.figure(figsize=(16, 4), dpi=200)
fig.patch.set_visible(False)
plt.axis('off')
# Wrap each image in a list to create a list of sequences of artists
imgs = [[plt.imshow(img, animated=True)] for img in imgs]

ani = Animation.ArtistAnimation(fig, imgs, interval=250, blit=True, repeat_delay=1000)

# Save the animation to a file
ani.save('buoyancygraph.gif', dpi=200)


2023-09-13 17:15:00,163 matplotlib.animation 0/1 INFO :: Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
2023-09-13 17:15:00,167 matplotlib.animation 0/1 INFO :: MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 3200x800 -pix_fmt rgba -r 4.0 -i pipe: -filter_complex 'split [a][b];[a] palettegen [p];[b][p] paletteuse' -y buoyancygraph.gif
